<a href="https://colab.research.google.com/github/ZeusKwon/CSAI_assignment/blob/main/n222a_%EC%8A%A4%ED%84%B0%EB%94%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *DATA SCIENCE / SECTION 2 / SPRINT 2 / NOTE 2*

# 📝 Assignment
---


### 1)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

url_train = "https://raw.githubusercontent.com/ZeusKwon/data-drive/main/prediction-of-h1n1-vaccination/train.csv"
url_train_labels="https://raw.githubusercontent.com/ZeusKwon/data-drive/main/prediction-of-h1n1-vaccination/train_labels.csv"
url_test = "https://raw.githubusercontent.com/ZeusKwon/data-drive/main/prediction-of-h1n1-vaccination/test.csv"
url_submission = "https://raw.githubusercontent.com/ZeusKwon/data-drive/main/prediction-of-h1n1-vaccination/submission.csv"

target = 'vacc_h1n1_f'

df = pd.merge(pd.read_csv(url_train),
                 pd.read_csv(url_train_labels)[target],
                left_index=True, right_index=True)
test = pd.read_csv(url_test)
submission = pd.read_csv(url_submission)

In [ ]:
train, val = train_test_split(df, train_size=0.80, test_size=0.20, stratify=df[target], random_state=2)

train.shape, val.shape, test.shape

((33723, 39), (8431, 39), (28104, 38))

# 특성공학
### 피처엔지니어링
- n_people_r -> 가정의 사람 수 (household_children + n_adult_r) 두 피처 제거
- employment_status -> 고용상태를 employed와 unemployed로만 나눔  

### Ordinal Encoder 필요  

- **h1n1_concern** -> H1N1 독감에 대한 우려 수준입니다. * 0 = 전혀 우려되지 않음, 1 = 별로 우려되지 않음, 2 = 다소 우려됨, 3 = 매우 우려됩니다.

- **h1n1_knowledge** -> H1N1 독감에 대한 지식 수준입니다. * 0 = 지식 없음, 1 = 약간의 지식, 2 = 많은 지식입니다

- **opinion_h1n1_vacc_effective** -> 백신효과에 대한 의견
0 = Not at all effective; 1 = Not very effective; 2 = Don't know; 3 = Somewhat effective; 4 = Very effective

### one-hot Encoding 필요  

- **raceeth4_i** -> 응답자의 인종 1 = Hispanic; 2 = Non-Hispanic, Black Only; 3 = Non-Hispanic, White Only; 4 = Non-Hispanic, Other or Multiple Races

- **sex_i** -> 응답자의 성별 1 = Male; 2 = Female 2

- **inc_pov** - 2008년 인구 조사 빈곤 임계값에 대한 응답자의 가구 연간 소득. 1 = > $75,000; 2 : <= $75,000; 3 : Below Poverty; 4 : Unknown

- **marital** -> 응답자의 결혼 여부 1 = Married; 2 = Not Married

- **rent_own_r** -> 응답자의 주거 상황 1 = Home is Owned; 2 = Home is Rented or Other Arrangement

In [ ]:
# employment_status -> 고용상태를 employed와 unemployed로만 나눔
fixed_data = []
for i in df["employment_status"]:
    if i == "Not in Labor Force":
        fixed_data.append("Unemployed")
    else:
        fixed_data.append(i)
df["employment_status"] = fixed_data

# n_people_r -> 가정의 사람 수 (household_children + n_adult_r)
df.drop(columns = ['household_children', 'n_adult_r'], inplace = True) 

In [ ]:
import numpy as np

def engineer(df):

    # 높은 카디널리티를 가지는 특성을 제거합니다.
    selected_cols = df.select_dtypes(include=['number', 'object'])
    labels = selected_cols.nunique() # 특성별 카디널리티 리스트
    selected_features = labels[labels <= 30].index.tolist() # 카디널리티가 30보다 작은 특성만 선택합니다.
    df = df[selected_features]
    
    # 새로운 특성을 생성합니다.
    behaviorals = [col for col in df.columns if 'behavioral' in col] 
    df['behaviorals'] = df[behaviorals].sum(axis=1)
    
    
    dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
    df.drop(columns=dels, inplace=True)
        
    return df


train = engineer(train)
val = engineer(val)
test = engineer(test)

In [ ]:
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

# n_people_r -> 가정의 사람 수 (household_children + n_adult_r)
X_train.drop(columns = ['household_children', 'n_adult_r'], inplace = True) 
X_test.drop(columns = ['household_children', 'n_adult_r'], inplace = True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
!pip install category_encoders

# DecisionTree 모델 구현

In [ ]:
# state제거 후 OrdinalEncoder 정확도
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

pipe = make_pipeline(
    OrdinalEncoder(handle_missing="value"),
    SimpleImputer(), 
    DecisionTreeClassifier(max_depth=9, random_state=2)
)

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print('훈련 정확도', pipe.score(X_train, y_train))
print('검증 정확도', pipe.score(X_val, y_val))
#훈련 정확도 0.8288111971058328
#검증 정확도 0.8257620685565176

훈련 정확도 0.8409690715535392
검증 정확도 0.8223223816866326


In [ ]:
# status 제거하고 ordinal 후 f값
from sklearn.metrics import f1_score

y_valid_pred = pipe.predict(X_val)
f1_score(y_val, y_valid_pred)
# 0.6131428983625479

0.5682997118155619

In [ ]:
# state살리고 후 OrdinalEncoder 정확도
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier

pipe = make_pipeline(
    OrdinalEncoder(handle_missing="value"),
    SimpleImputer(), 
    DecisionTreeClassifier(max_depth=9, random_state=2)
)

pipe.fit(X_train, y_train)
y_pred = pipe.predict(X_test)
print('훈련 정확도', pipe.score(X_train, y_train))
print('검증 정확도', pipe.score(X_val, y_val))
#simple
#훈련 정확도 0.8288111971058328
#검증 정확도 0.8257620685565176

#ordinal
#훈련 정확도 0.8419179788275064
#검증 정확도 0.8137824694579527

훈련 정확도 0.8409690715535392
검증 정확도 0.8223223816866326


In [ ]:
# status 살리고 ordinal 후 f값
from sklearn.metrics import f1_score

y_valid_pred = pipe.predict(X_val)
f1_score(y_val, y_valid_pred)

# 제거보다 더 떨어짐

0.5682997118155619

## 하이터 파라미터 조정

In [ ]:
#이전에 썼던 train & val 합쳐서 다시 전처리
train1 = engineer(df)

X_train1 = train1[features]
y_train1 = train1[target]
# n_people_r -> 가정의 사람 수 (household_children + n_adult_r)
X_train1.drop(columns = ['household_children', 'n_adult_r'], inplace = True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
from category_encoders import OneHotEncoder
pipe = make_pipeline(        
        OneHotEncoder(use_cat_names=True), SimpleImputer(), 
        DecisionTreeClassifier(random_state=1, class_weight='balanced', splitter='best', min_samples_split=450, min_samples_leaf=3, max_depth=19)
        )        
pipe.fit(X_train1, y_train1)            
y_tr_pred = pipe.predict(X_train1)        
print('훈련 정확도', pipe.score(X_train1, y_train1))

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


훈련 정확도 0.7692982872325284


In [ ]:
#train + val
from sklearn.metrics import f1_score

y_tr_pred = pipe.predict(X_train1)
f1_score(y_train1, y_tr_pred)
# 0.6095787560262388

0.6117609485408598

In [ ]:
from category_encoders import OneHotEncoder
pipe = make_pipeline(        
        OrdinalEncoder(handle_missing="value"), SimpleImputer(), 
        DecisionTreeClassifier(random_state=1, class_weight='balanced', min_samples_split=460, max_depth=18)
        )        
pipe.fit(X_train1, y_train1)            
y_tr_pred = pipe.predict(X_train1)        
print('훈련 정확도', pipe.score(X_train1, y_train1))
# 훈련 정확도 0.7799971532950609

훈련 정확도 0.776415049580111


In [ ]:
#train + val Ordinal
from sklearn.metrics import f1_score

y_tr_pred = pipe.predict(X_train1)
f1_score(y_train1, y_tr_pred)
# 0.6174092409240924

0.6150390066576809

### 2)

# 랜덤포레스트 구현

In [ ]:
%%time
from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OneHotEncoder(use_cat_names=True), 
    SimpleImputer(), 
    RandomForestClassifier(n_jobs=-1, random_state=10, oob_score=True)
)

pipe.fit(X_train, y_train)
print('검증 정확도: ', pipe.score(X_val, y_val))
# 검증 정확도:  0.8225596014707627

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


검증 정확도:  0.8225596014707627
CPU times: user 7.82 s, sys: 46.2 ms, total: 7.87 s
Wall time: 4.73 s


In [ ]:
# OneHotEncoder
from sklearn.metrics import f1_score

y_valid_pred = pipe.predict(X_val)
f1_score(y_val, y_valid_pred)
# 0.5676156583629893

0.556872037914692

In [ ]:
%%time
from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(), 
    RandomForestClassifier(n_jobs=-1, random_state=10, oob_score=True)
)

pipe.fit(X_train, y_train)
print('검증 정확도: ', pipe.score(X_val, y_val))

검증 정확도:  0.8239829201755426
CPU times: user 6.71 s, sys: 65.3 ms, total: 6.78 s
Wall time: 4.05 s


In [ ]:
# OrdinalEncoder
from sklearn.metrics import f1_score

y_valid_pred = pipe.predict(X_val)
f1_score(y_val, y_valid_pred)

0.5540865384615384

#하이퍼 파라미터 조정

In [ ]:
%%time
from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(
    OneHotEncoder(use_cat_names=True), 
    SimpleImputer(), 
    RandomForestClassifier(n_jobs=-1, random_state=10, oob_score=True,class_weight='balanced', min_samples_split=460, max_depth=18)
)

pipe.fit(X_train, y_train)
print('검증 정확도: ', pipe.score(X_val, y_val))

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


검증 정확도:  0.7630174356541336
CPU times: user 4.18 s, sys: 64.4 ms, total: 4.25 s
Wall time: 2.79 s


In [ ]:

from sklearn.metrics import f1_score

y_valid_pred = pipe.predict(X_val)
f1_score(y_val, y_valid_pred)
# 0.6103744149765991

0.6103744149765991

**디시전 트리보다 랜덤포레스트의 f-score가 더 낮게 나온다.**
 
개인적인 생각
- 랜덤포레스트에서 학습할때 중요도 비교를 스스로 해서 특성의 값을 비교하는데, 특성에 nan값이 많이 포함되어있는 경우 불순도가 높지만 중요한 노드로 인식이 되기 때문에 f-score값이 더 낮아진것 같다.

# train + val값 더해서 최종 랜덤포레스트 f스코어 출력

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

url_train = "https://raw.githubusercontent.com/ZeusKwon/data-drive/main/prediction-of-h1n1-vaccination/train.csv"
url_train_labels="https://raw.githubusercontent.com/ZeusKwon/data-drive/main/prediction-of-h1n1-vaccination/train_labels.csv"
url_test = "https://raw.githubusercontent.com/ZeusKwon/data-drive/main/prediction-of-h1n1-vaccination/test.csv"
url_submission = "https://raw.githubusercontent.com/ZeusKwon/data-drive/main/prediction-of-h1n1-vaccination/submission.csv"

target = 'vacc_h1n1_f'

df = pd.merge(pd.read_csv(url_train),
                 pd.read_csv(url_train_labels)[target],
                left_index=True, right_index=True)
test = pd.read_csv(url_test)
submission = pd.read_csv(url_submission)

In [ ]:
# employment_status -> 고용상태를 employed와 unemployed로만 나눔
fixed_data = []
for i in df["employment_status"]:
    if i == "Not in Labor Force":
        fixed_data.append("Unemployed")
    else:
        fixed_data.append(i)
df["employment_status"] = fixed_data

In [ ]:
#이전에 썼던 train & val 합쳐서 다시 전처리
features = train.drop(columns=[target]).columns
train1 = engineer(df)

In [ ]:
import numpy as np

def engineer(df):

    # 높은 카디널리티를 가지는 특성을 제거합니다.
    selected_cols = df.select_dtypes(include=['number', 'object'])
    labels = selected_cols.nunique() # 특성별 카디널리티 리스트
    selected_features = labels[labels <= 30].index.tolist() # 카디널리티가 30보다 작은 특성만 선택합니다.
    df = df[selected_features]
    
    # 새로운 특성을 생성합니다.
    behaviorals = [col for col in df.columns if 'behavioral' in col] 
    df['behaviorals'] = df[behaviorals].sum(axis=1)
    
    
    dels = [col for col in df.columns if ('employment' in col or 'seas' in col)]
    df.drop(columns=dels, inplace=True)
        
    return df


train1 = engineer(train1)
test1 = engineer(test)

In [ ]:
X_train1 = train1[features]
y_train1 = train1[target]
# n_people_r -> 가정의 사람 수 (household_children + n_adult_r)
X_train1.drop(columns = ['household_children', 'n_adult_r'], inplace = True) 

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
%%time
from category_encoders import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer 
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(        
        OneHotEncoder(use_cat_names=True), SimpleImputer(), 
        DecisionTreeClassifier(random_state=1, class_weight='balanced', splitter='best', min_samples_split=450, min_samples_leaf=3, max_depth=19)
        )        
pipe.fit(X_train1, y_train1)            
y_tr_pred = pipe.predict(X_train1)        
print('훈련 정확도', pipe.score(X_train1, y_train1))

/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


훈련 정확도 0.7692982872325284
CPU times: user 937 ms, sys: 26 ms, total: 963 ms
Wall time: 978 ms


In [ ]:
#train + val
from sklearn.metrics import f1_score

y_tr_pred = pipe.predict(X_train1)
f1_score(y_train1, y_tr_pred)
# 0.6117609485408598

0.6117609485408598

In [ ]:
X_train1.shape

(42154, 29)

## 🔥 

In [5]:
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from category_encoders import OneHotEncoder

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split

url_train = "https://raw.githubusercontent.com/ZeusKwon/data-drive/main/prediction-of-h1n1-vaccination/train.csv"
url_train_labels="https://raw.githubusercontent.com/ZeusKwon/data-drive/main/prediction-of-h1n1-vaccination/train_labels.csv"
url_test = "https://raw.githubusercontent.com/ZeusKwon/data-drive/main/prediction-of-h1n1-vaccination/test.csv"
url_submission = "https://raw.githubusercontent.com/ZeusKwon/data-drive/main/prediction-of-h1n1-vaccination/submission.csv"

target = 'vacc_h1n1_f'

df = pd.merge(pd.read_csv(url_train),
                 pd.read_csv(url_train_labels)[target],
                left_index=True, right_index=True)
test = pd.read_csv(url_test)
submission = pd.read_csv(url_submission)

In [39]:
train, val = train_test_split(df, train_size=0.80, test_size=0.20, stratify=df[target], random_state=2)

train.shape, val.shape, test.shape

((33723, 39), (8431, 39), (28104, 38))

In [34]:
# 중복샘플 제거
train.drop_duplicates(inplace = True)
val.drop_duplicates(inplace = True)

In [40]:
# 특성 엔지니어링
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def engineer(df):
    
    # behaviorals 관련 새로운 특성 생성
    behaviorals = [col for col in df.columns if 'behavioral' in col] 
    df['behaviorals'] = df[behaviorals].sum(axis=1)
    

    # 타겟 변수와 관계 없는 seasonal flu관련 특성 제거
    dels = [col for col in df.columns if 'seas' in col]
    df.drop(columns=dels, inplace=True)
        
    return df

train = engineer(train)
val = engineer(val)
test = engineer(test)

In [41]:
# 훈련/검증/테스트 데이터를 특성과 타겟으로 분리
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [47]:
from category_encoders import TargetEncoder

pipe = make_pipeline(
    TargetEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(n_jobs=-1, random_state=2, oob_score=True, 
                           n_estimators=450, max_depth=11, class_weight='balanced')
)

pipe.fit(X_train, y_train)

y_val_pred = pipe.predict(X_val)
print('f1_score', f1_score(y_val, y_val_pred))
# 0.6295484436650592

f1_score 0.6295484436650592


**원핫인코딩과 타겟인코딩은 둘 다 범주형 기능을 인코딩하는데 사용한다.  
원핫인코딩은 보유하고 있는 범주의 수에 따라서 높은 차원을 만들게 될 수 있다.  
이 성질은 데이터세트가 크면 클수록 문제가 된다
하지만 타겟인코딩은 데이터 세트의 차원을 추가하지 않고 인코딩을 수행합니다.  
따라서 좋은 첫 번째 시도 인코딩 방법으로 사용할 수 있습니다.**

## 참고자료

- [Random Forests for Complete Beginners](https://victorzhou.com/blog/intro-to-random-forests/)